# Table of Contents

01 Importing Libraries <br />
02 Importing Data <br />
03 Performing Aggregations to Entire Dataframe <br />
04 Creating a Loyalty Flag <br />
05 Creating a Spending Flag <br />
06 Creating a Frequency Flag <br />
07 Exporting Data <br />

# 01 Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os

# 02 Importing Data

In [2]:
# Create basic path
path = r'C:\Users\Rajit\Python\11-2023 Instacart Basket Analysis'

In [3]:
# Import "orders_products_merged_v2.csv" data set into notebook 
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_v2.pkl'))

# 03 Performing Aggregations to Entire Dataframe

1. In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe

In [5]:
ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


In [6]:
ords_prods_merge.groupby('department_id').order_number.mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

The results for the whole dataset were slightly different compared to the subset of 1 million. Some of the department id's yielded smaller values for the full dataset (like 1,4,8), but the other yielded larger values. 
Likely because we have more points to influence the mean for each department id.

Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

# 04 Creating a Loyalty Flag

In [7]:
#Creating Seperate Column for Max order
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform('max')

In [8]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

C:\Users\Rajit\AppData\Local\Temp\ipykernel_10536\3647449420.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Loyal customer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'


In [9]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [10]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [11]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [12]:
# Performing Multiple Aggregations to group prices by customer type
ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:

If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”

If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

# 05 Creating a Spending Flag

In [13]:
#Creating an average spent Column
ords_prods_merge['avg_spent'] = ords_prods_merge.groupby(['user_id'])['prices'].transform('mean')

In [14]:
ords_prods_merge.head(10)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,price_range,Store_Actvitiy,busiest_period_of_day,max_order,loyalty_flag,avg_spent
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,Mid-range product,Regular Days,Most_Orders,32,Regular customer,6.935811
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,Mid-range product,Regular Days,Average_Orders,32,Regular customer,6.935811
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,Mid-range product,Busiest Days,Average_Orders,5,New customer,7.930208
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,10,0,Mid-range product,Slowest Days,Most_Orders,3,New customer,4.972414
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,Mid-range product,Slowest Days,Average_Orders,3,New customer,4.972414
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,26.0,7,0,Mid-range product,Busiest Days,Average_Orders,26,Regular customer,6.935398
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,30.0,2,0,Mid-range product,Regular Days,Most_Orders,9,New customer,5.957576
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,12,3,10,30.0,1,0,Mid-range product,Slowest Days,Most_Orders,12,Regular customer,6.680000
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,10,1,20,19.0,1,0,Mid-range product,Busiest Days,Average_Orders,20,Regular customer,7.162500
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,15,1,12,15.0,2,1,Mid-range product,Busiest Days,Most_Orders,20,Regular customer,7.162500


In [15]:
#Classifying Low spender as person who's avg spent is less than 10
ords_prods_merge.loc[ords_prods_merge['avg_spent'] < 10, 'spending_flag'] = 'Low Spender'

C:\Users\Rajit\AppData\Local\Temp\ipykernel_10536\1151216812.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Low Spender' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['avg_spent'] < 10, 'spending_flag'] = 'Low Spender'


In [16]:
#Classifying High Spender as person who's avg spent is greater than or equal to 10
ords_prods_merge.loc[ords_prods_merge['avg_spent'] >= 10, 'spending_flag'] = 'High Spender'

In [17]:
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low Spender     31770614
High Spender      634245
Name: count, dtype: int64

In [18]:
ords_prods_merge.head(10)

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered,price_range,Store_Actvitiy,busiest_period_of_day,max_order,loyalty_flag,avg_spent,spending_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,3.0,5,0,Mid-range product,Regular Days,Most_Orders,32,Regular customer,6.935811,Low Spender
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,20.0,1,1,Mid-range product,Regular Days,Average_Orders,32,Regular customer,6.935811,Low Spender
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,6.0,20,0,Mid-range product,Busiest Days,Average_Orders,5,New customer,7.930208,Low Spender
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,NaN,10,0,Mid-range product,Slowest Days,Most_Orders,3,New customer,4.972414,Low Spender
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,9.0,11,1,Mid-range product,Slowest Days,Average_Orders,3,New customer,4.972414,Low Spender
5,1,Chocolate Sandwich Cookies,61,19,5.8,1701441,777,16,1,7,26.0,7,0,Mid-range product,Busiest Days,Average_Orders,26,Regular customer,6.935398,Low Spender
6,1,Chocolate Sandwich Cookies,61,19,5.8,1871483,825,3,2,14,30.0,2,0,Mid-range product,Regular Days,Most_Orders,9,New customer,5.957576,Low Spender
7,1,Chocolate Sandwich Cookies,61,19,5.8,1290456,910,12,3,10,30.0,1,0,Mid-range product,Slowest Days,Most_Orders,12,Regular customer,6.680000,Low Spender
8,1,Chocolate Sandwich Cookies,61,19,5.8,369558,1052,10,1,20,19.0,1,0,Mid-range product,Busiest Days,Average_Orders,20,Regular customer,7.162500,Low Spender
9,1,Chocolate Sandwich Cookies,61,19,5.8,589712,1052,15,1,12,15.0,2,1,Mid-range product,Busiest Days,Most_Orders,20,Regular customer,7.162500,Low Spender


In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:

If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”

If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”

If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.

# 06 Creating a Frequency Flag

In [19]:
#Creating an order frequency column
ords_prods_merge['median_order_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform('median')

In [20]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_dow,order_hour_of_day,...,add_to_cart_order,reordered,price_range,Store_Actvitiy,busiest_period_of_day,max_order,loyalty_flag,avg_spent,spending_flag,median_order_frequency
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,5,0,Mid-range product,Regular Days,Most_Orders,32,Regular customer,6.935811,Low Spender,8.0
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,1,1,Mid-range product,Regular Days,Average_Orders,32,Regular customer,6.935811,Low Spender,8.0
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,20,0,Mid-range product,Busiest Days,Average_Orders,5,New customer,7.930208,Low Spender,8.0
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,10,0,Mid-range product,Slowest Days,Most_Orders,3,New customer,4.972414,Low Spender,9.0
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,11,1,Mid-range product,Slowest Days,Average_Orders,3,New customer,4.972414,Low Spender,9.0


In [21]:
#Classifying Non-Frequent Customer as person's median order greater than 20
ords_prods_merge.loc[ords_prods_merge['median_order_frequency'] > 20, 'frequency_flag'] = 'Non-Frequent Customer'

C:\Users\Rajit\AppData\Local\Temp\ipykernel_10536\2844540773.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Non-Frequent Customer' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods_merge.loc[ords_prods_merge['median_order_frequency'] > 20, 'frequency_flag'] = 'Non-Frequent Customer'


In [22]:
#Classifying Regular Customer as person's median order between 10 and 20
ords_prods_merge.loc[(ords_prods_merge['median_order_frequency'] <= 20) & (ords_prods_merge['median_order_frequency'] > 10), 'frequency_flag'] = 'Regular customer'

In [23]:
#Classifying Frequent Customer as person's median order less than 10
ords_prods_merge.loc[ords_prods_merge['median_order_frequency'] <= 10, 'frequency_flag'] = 'Frequent Customer'

In [24]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent Customer        21559853
Regular customer          7208564
Non-Frequent Customer     3636437
NaN                             5
Name: count, dtype: int64

# 07 Exporting Data

In [41]:
#Export ords_prods_merge in pkl format
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_v48.pkl'))